In [1]:
import glob
import shutil
import cv2
import os
from PIL import Image
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import * 

In [3]:

def relu(x):
    return tf.nn.relu(x)

def softmax(x):
    return tf.nn.softmax(x)

# Build convolutional block
def conv_block(inputs, filters, kernel_size, strides):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = relu(x)
    return x

# Build bottleneck block
def bottleneck_block(inputs, filters, kernel_size, strides, expansion_factor):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, in_channels * expansion_factor, 1, 1)
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = relu(x)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.add([x, inputs])
    return x

# Build MobileNetV2 architecture
def mobilenetv2(input_shape, num_classes, expansion_factor=6):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = conv_block(inputs, 32, 3, 2)
    x = bottleneck_block(x, 16, 3, 1, expansion_factor)
    x = bottleneck_block(x, 24, 3, 2, expansion_factor)
    x = bottleneck_block(x, 24, 3, 1, expansion_factor)
    x = bottleneck_block(x, 32, 3, 2, expansion_factor)
    x = bottleneck_block(x, 32, 3, 1, expansion_factor)
    x = bottleneck_block(x, 32, 3, 1, expansion_factor)
    x = bottleneck_block(x, 64, 3, 2, expansion_factor)
    x = bottleneck_block(x, 64, 3, 1, expansion_factor)
    x = bottleneck_block(x, 64, 3, 1, expansion_factor)
    x = bottleneck_block(x, 64, 3, 1, expansion_factor)
    x = bottleneck_block(x, 96, 3, 1, expansion_factor)
    x = bottleneck_block(x, 96, 3, 1, expansion_factor)
    x = bottleneck_block(x, 96, 3, 1, expansion_factor)
    x = bottleneck_block(x, 160, 3, 2, expansion_factor)
    x = bottleneck_block(x, 160, 3, 1, expansion_factor)
    x = bottleneck_block(x, 160, 3, 1, expansion_factor)
    x = bottleneck_block(x, 320, 3, 1, expansion_factor)
    x = conv_block(x, 1280, 1, 1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation=softmax)(x)
    return tf.keras.Model(inputs, outputs)

# Define loss function
def loss_fn(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred)

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model = mobilenetv2(input_shape=(224,224,3), num_classes=10)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train the model
model.fit(train_data, epochs=10, validation_data=val_data)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')

# Save the model
model.save('mobilenetv2.h5')




NameError: name 'train_data' is not defined